In [ ]:
import requests
import json
import pprint
import zipfile

In [ ]:
r = requests.get('https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip', auth=('user','pass'), verify = False)
open('fgdb_elev_30m_digital_elevation_model.zip', 'wb').write(r.content)

with zipfile.ZipFile('fgdb_elev_30m_digital_elevation_model.zip', 'r') as zip_ref:
    zip_ref.extractall(r'C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab3\GIS 5571 Lab3')  # "r" is used to pass as raw string

In [ ]:
# This cell clips both of the rasters to the study area using a "mask" polygon created using the GUI

arcpy.management.Clip("Multispectral_MTD_MSIL1C", "562425.7716 4869457.1349 588616.4866 4889162.7259", r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab3\GIS 5571 Lab 3\GIS 5571 Lab 3.gdb\Multispectral_MTD_MSIL1C_Cli", "Mask", "-1", "ClippingGeometry", "MAINTAIN_EXTENT")

arcpy.management.Clip(r"Digital Elevation Model - 30m Resolution\Digital Elevation Model", "562426.367088914 4869456.26120291 588617.078055312 4889161.84915998", r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab3\GIS 5571 Lab 3\GIS 5571 Lab 3.gdb\DigitalElevationModel_Clip", "Mask", "32767", "ClippingGeometry", "MAINTAIN_EXTENT")

In [ ]:
# This cell begins the processing for the cost surface creation

# First the first derivitave of the DEM is taken to obtain the slope
arcpy.ddd.Slope("DigitalElevationModel_Clip", r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab3\GIS 5571 Lab 3\GIS 5571 Lab 3.gdb\Slope_Digita1", "DEGREE", 1, "PLANAR", "METER")
# The slope raster can then be reclassifed to reflect the cost assosiated with movement
arcpy.ddd.Reclassify("Slope_Digita1", "VALUE", "0 4.698385 1;4.698385 9.984069 2;9.984069 15.563401 3;15.563401 21.730032 4;21.730032 28.777610 5;28.777610 36.412486 6;36.412486 44.341011 7;44.341011 51.975888 8;51.975888 58.729817 9;58.729817 74.880516 10", r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab3\GIS 5571 Lab 3\GIS 5571 Lab 3.gdb\Reclass_Slop1", "DATA")

# Next the image is classified using suppervised pixel based random trees, this allows for much more accurate assesment when compared to unsupervised. The training samples where generated using the classification wizard tool
# After classification the landcover type can be given a cost based on how difficult it is to travel on, this is done by adding a cost field
arcpy.management.AddField("Reclassified_202112121841032960699", "Cost", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

# The slope raster and the classified raster are then combined to form a single cost surface
out_raster = arcpy.sa.Combine("Reclass_Slop1;Reclassified_202112121841032960699"); out_raster.save(r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab3\GIS 5571 Lab 3\GIS 5571 Lab 3.gdb\Combine_Recl1")

# The two cost attributes can then be added to reflect the overall cost of travel across each cell
arcpy.management.CalculateField("Combine_Recl1", "Cost", "!Reclass_Slop1! + !Reclassified_202112121841032960699!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")

# To change the weights of each attribute, you can simply change the math to include a multiplication of a percent to each the slope and land type. This is explained further in the write up

In [ ]:
# This cell generated a path of least cost given the cost surface, start point, and end point.
# The start and end points were generated based on the coordinates given within the prompt
# First the "Distance Accumulation" tool must be used to create the distance and back direction rsaters to feed into the "Optimal Path As Line" tool
out_distance_accumulation_raster = arcpy.sa.DistanceAccumulation("Source", None, None, "Reclass_Comb2", None, "BINARY 1 -30 30", None, "BINARY 1 45", r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab3\GIS 5571 Lab 3\GIS 5571 Lab 3.gdb\BackDir", None, None, None, None, None, '', "PLANAR"); out_distance_accumulation_raster.save(r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab3\GIS 5571 Lab 3\GIS 5571 Lab 3.gdb\Distanc_Sour7")
arcpy.sa.OptimalPathAsLine("Destination", "Distanc_Sour7", "BackDir", r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab3\GIS 5571 Lab 3\GIS 5571 Lab 3.gdb\Optimal_Destina5", "Goal", "EACH_ZONE", "DESTINATIONS_TO_SOURCES")

# These two tools can be reused for each cost surface created reflecting each of the parameter/preferance sets
